# Gas Storage

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

import sys
if '../..' not in sys.path:
    sys.path.append('../..')

from rivapy.models.ornstein_uhlenbeck import OrnsteinUhlenbeck
from rivapy.models.gbm import GeometricBrownianMotion
from rivapy.instruments.gasstorage_specification import GasStorageSpecification
from rivapy.pricing.gas_storage_pricer import PricingParameter, pricing_lsmc

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Gas Storage Valuation Using a Monte Carlo Method 

### Literature

[1] Gas Storage Valuation Using a Monte Carlo Method (Boogert and De Jong)

[2] Valuing American Options by Simulation: A Simple Least-Squares Approach (Longstaff and Schwartz)

A Least Squares Monte Carlo (LSMC) valuation method is used to adjust storage trading decisions to price conditions.

The task is to find the optimal operation (injection and withdrawal) of the storage, depending on current and expected gas prices, based on predictable price movements (seasonality) and unpredictable price fluctuations (volatility).
The storage value thus depends on operational constraints such as working volume (effective capacity) and injection and withdrawal rates (flexibility) as well as the dynamics of market prices, which include spot price volatility, mean reversion(!) and seasonality.

For the intrinsic value approach, calculate the optimal position given the available forward curve and take this position on the forward market.
Additional value (extrinsic value) can be created by reacting to the price fluctuations on the spot market.
The storage operator must thus choose between operating on the forward market (forward-based valuation) and operating on the spot market (spot-based valuation), or a combination of these.

**The storage contract** 

Consider a storage contract from the perspective of the contract holder. It is signed at $t=0$ and settled at $t=T+1$.
The contract allows the holder to take a desired action at any discrete date $t = 1, ..., T$ after the spot price $S(t)$ is revealed.
Every day the holder of a storage contract can choose to inject gas, do nothing, or withdraw gas, within certain volumetric limitations.

An injection at day $t$ is a positive volume change $\Delta v(t)$ and represents costs, while a withdrawal is a negative volume change $\Delta v(t)$ and represents profits. Note that at time $t = 0$ the holder cannot take an action, and $\Delta v(0) = 0$.
We assume that the market has incorporated all knowledge about the optimal usage of storage into the prevailing forward curve and that the market will not be influenced by our trades.

We denote the (accumulated) volume in storage at the start of day $t$ by $v(t)$ and the volume change with $\Delta v(t) = v(t+1) - v(t)$, the volume at day $t$ is 
$$v(t) = v(0) + \sum_{i=1}^t \Delta v(i-1) $$

The payoff at day $t=0, ..., T$ is 
$$ h(S(t), \Delta v(t)) = \begin{cases}
-c(S(t)) \Delta v(t)  &\text{inject at day } t \newline
0  &\text{do nothing at day } t \newline
-p(S(t)) \Delta v(t)  &\text{withdraw at day } t
\end{cases} $$

where $c(S(t)) \geq 0$ and $p(S(t)) \geq 0$ represent the cost of injection and profit of withdrawal, respectively, 

$$ c(S(t)) = (1+a_1)S(t) + b_1 \ge 0$$
$$ p(S(t)) = (1-a_2)S(t) - b_2 \ge 0$$

with transaction costs $a$ and bid-ask spreads $b$.

We denote the interest rate by $\delta$.
At settlement $T+1$ the holder receives a potential penalty, denoted by $q(S(T + 1),v(T+ 1))$, depending on the remaining gas in the storage, current price level, ...

The volume in storage must stay between a minimum level and a maximum level for $t=1,...,T+1$:
$$0 \le v^{min}(t) \le v(t) \le v^{max}(t)$$ 

The injection and withdrawal is limited per day:
$$ i^{min}(t, v(t)) \le \Delta v(t) \le i^{max}(t, v(t))$$

**Valuation of a Storage Contract**

The value of a storage contract is the expected value of the accumulated future payoffs $h(S(t), Av(t))$ under the most optimal strategy $\pi = \{\pi(t, S(t), v(t))\}_{t=1}^T $, which is dependent on the volume.

The set of allowed volume levels at day $t$ is denoted by 

$$\mathcal{V}(t) = \{ v | v^{min} (t) \leq v \leq v^{max} (t)\}$$

and the set of all allowed actions on day $t$ being at volume $v(t)$ by

$$\mathcal{D}(t, v(t)) = \{ \Delta v | v^{min} (t+1) \leq v(t) -\Delta v \leq v^{max} (t+1),  i^{min} (t, v(t)) \leq \Delta v \leq i^{max} (t, v(t))\} $$

The value of a storage contract starting on day $t$ at volume level $v(t)$ is denoted by $U(t, S(t), v(t))$ and the continuation value, which is the value attached to the contract after taking an allowed action $\Delta v \in \mathcal{D}(t, v(t))$, is defined as

$$ C(t, S(t), v(t), \Delta v) = E [e^{-\delta} U(t+1, S(t+1), v(t)+\Delta v)] $$

Hence, $U(t, S(t), v(t))$ satisfies the following dynamic program:

$$ U(T, S(T+1), v(T+1)) = q(S(T+1), v(T+1))$$
$$ U(t, S(t), v(t)) = \max_{\Delta v \in \mathcal{D}(t,v(t))} \{ h(S(t), \Delta v) + C(t, S(t), v(t), \Delta v)\} \quad \forall t$$

The holder of the contract weighs different actions against one another on the basis of their direct payoff plus expected future payoff and thus needs to calculate the continuation values for all $t$ for all possible actions for allowed volume levels to make a decision as the dynamic program is solved backward in time.

To find an approximate solution, a regression-based method like Least Squares Monte Carlo may be used.

**Pricing Algorithm**

1. Simulate $M$ independent price paths $S^b(1), ..., S^b(T+1)$ for $b=1, ..., M$ starting at a given $S(0)$.

2. Assign a value to the contract at maturity according to 

    $$Y^b(T+1,S^b(T+1), v(T+1)) = q(S^b(T+1),v(T+1))$$ 

    (The continuation values are unknown and approximated for all $M$ independent paths from the simulation by
    
    $$ C^b(t, S^b(t), v(t+1,n)) \approx e^{-\delta} Y^b(t+1, S(t+1), v(t+1,n)) $$
    
    where $Y^b(t+1, S(t+1), v(t+1,n))$ is the accumulated value of future realized cash flows in path $b=1, ..., M$ following optimal decisions starting at time $t+1$ being at volume level $v(t+1)$ and price $S^b (t+1)$. 
    
    At day $T+1$, the continuation value is equal to the penalty function.)

3. Apply backward induction for $t=T, ..., 1$ (the decision rule for all points in the time-volume grid is determined by stepping backwards).

    For each $t$, step over $N$ allowed volume levels $v(t,n) \in \mathcal{V}(t)$:

- Run an OLS regression to find an approximation of the continuation value $C^b(t, S^b(t), v(t+1,n))$ according to 

    $$ C(t, S(t), v(t), \Delta v) \approx \sum_{q=1}^Q \phi _q (t,S(t),\mathcal{V}(t), \Delta (v)) \beta _{q,t} $$
    
    (The best regression coefficients $\hat{\beta}$ may be estimated by an ordinary least squares regression and the continuation values can be approximated by a finite linear combination of known basis functions $\phi _q(S(t))$.)

- Combine the different continuation values $\hat{C}$ into a decision rule $\hat{\pi}^b (t, S^b(t), v(t))$, according to 

    $$ \hat{\pi}^b (t, S^b(t), v(t)) = \arg \max_{\Delta v \in \mathcal{D}(t,v(t))} \{ h(S^b(t), \Delta v) + \hat{C}^b(t, S^b(t), v(t) + \Delta v)\} $$

    (The decision for all allowed volume levels that is expected to have the highest accumulated payoff. This is checked against the given constraints to find achievable actions.)

- Implement the decision rule to calculate the accumulated future cash flows $Y^b(t, S^b(t), v(t))$ according to 

    $$ \hat{Y}^b(t+1,S^b(t+1), v(t+1)) = h(S^b(t+1), \hat{\pi}^b(t+1,S^b(t+1),v(t+1))) + e^{-\delta} Y^b(t+2, S^b(t+2), v(t+2) + \hat{\pi}^b(t+1,S^b(t+1),v(t+1))) $$
   
4. Storage value is the average accumulated future cash flow over all price paths according to 

    $$ \hat{U}(0, S(0), v(0)) = \frac{1}{M} \sum_{b=1}^M Y^b(1,S^b(1),v(0)) $$

    (at the valuation date $t=0$, the value of the storage contract is set equal to the average value of the future accumulated cash flows)

### Code

In [ ]:
def create_contract_dates(startdate: dt.datetime, enddate: dt.datetime, datestep:dt.timedelta)->list:
    dates=[startdate]
    while dates[-1] <= enddate-datestep:
        dates.append(dates[-1]+datestep)

    #dates=[startdate]*n #nb_timesteps 
    #for i in range(1,n):
    #    dates[i] = dates[i-1] + dateStep

    return dates

In [ ]:
## Setting the parameters
num_sims = 1000 #number of independent price paths simulated
S0 = 1.0 #starting value

n_vol_levels = 101 #101
storage_capacity = 1.0 #1_000
max_withdrawal = -0.1 #-7500
max_injection = 0.1 #2500

In [ ]:
startdate = dt.datetime.fromisoformat('2022-01-01')
enddate = dt.datetime.fromisoformat('2022-12-31')
dateStep = dt.timedelta(days=1) #daily nomination
contractdates = create_contract_dates(startdate, enddate, dateStep)
timegrid = np.array([(contractdates[i] - contractdates[0]).days/365.0 for i in range(len(contractdates))])
rnd = np.random.normal(size=(timegrid.shape[0], num_sims))

In [ ]:
# Simulate M independent price paths S^b(1), S^b(T+1) for b = 1...M starting at S(0)
gbm_sim = GeometricBrownianMotion(drift=0.2, volatility=0.01)
gbm = gbm_sim.simulate(timegrid, S0, rnd) 
ou_sim = OrnsteinUhlenbeck(speed_of_mean_reversion=0.1, volatility=0.1, mean_reversion_level=2.0)
ou = ou_sim.simulate(timegrid=timegrid, start_value=S0, rnd=rnd)

In [ ]:
path = ou
params = PricingParameter(n_time_steps = None, n_actions = None, n_vol_levels = n_vol_levels)
store = GasStorageSpecification(storage_capacity, max_withdrawal, max_injection)  

In [ ]:
cashflow, vol_levels = pricing_lsmc(store, params, path, num_sims)

In [ ]:
price = cashflow[0,0,0]
dispatch = np.mean(vol_levels, axis=1)    # 'mean timetable' of facility
print(price)

In [ ]:
plt.figure(figsize=(12,8))
#plt.plot(vol_levels, 'tab:blue', alpha=0.1)
plt.plot(dispatch, 'r')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(path, 'tab:blue', alpha=0.1)
plt.plot(np.mean(path, axis=1), 'r')

**Valuation of a Storage Contract**

The value of a storage contract is the expected value of the accumulated future payoffs $h(S(t), Av(t))$ under the most optimal strategy $\pi = \{\pi(t, S(t), v(t))\}_{t=1}^T $, which is dependent on the volume.

The pricing problem thus reads

$$ \sup_\pi E \left[ \sum_{i=0}^T e^{-\delta t} h(S(t), \Delta v(t)) + e^{-\delta (T+1)} q(S(T+1), v(T+1))\right] $$

under a risk-neutral pricing measure (for a unique arbitrage-free price, assume some level of market completeness / a risk premium to compensate for residual risk).

The dynamic program is solved backward in time, hence a decision rule for every time $t$ and all possible volume levels has to be found.

The set of allowed volumes levels at day $t$ is denoted by 

$$\mathcal{V}(t) = \{ v | v^{min} (t) \leq v \leq v^{max} (t)\}$$

and the set of all allowed actions on day $t$ being at volume $v(t)$ by

$$\mathcal{D}(t, v(t)) = \{ \Delta v | v^{min} (t+1) \leq v(t) -\Delta v \leq v^{max} (t+1),  i^{min} (t, v(t)) \leq \Delta v \leq i^{max} (t, v(t))\} $$

The value of a storage contract starting on day $t$ at volume level $v(t)$ is denoted by $U(t, S(t), v(t))$ and the continuation value, which is the value attached to the contract after taking an allowed action $\Delta v \in \mathcal{D}(t, v(t))$, is defined as

$$ C(t, S(t), v(t), \Delta v) = E [e^{-\delta} U(t+1, S(t+1), v(t)+\Delta v)] $$

Hence, $U(t, S(t), v(t))$ satisfies the following dynamic program:

$$ U(T, S(T+1), v(T+1)) = q(S(T+1), v(T+1))$$
$$ U(t, S(t), v(t)) = \max_{\Delta v \in \mathcal{D}(t,v(t))} \{ h(S(t), \Delta v) + C(t, S(t), v(t), \Delta v)\} \quad \forall t$$

The holder of the contract weighs different actions against one another on the basis of their direct payoff plus expected future payoff and thus needs to calculate the continuation values for all $t$ for all possible actions for allowed volume levels to make a decision.

To find an approximate solution, a regression-based method like Least Squares Monte Carlo may be used.

**Least Squares Monte Carlo**

The continuation values may be approximated by a finite linear combination of known basis functions $\phi _q(t,S(t),\mathcal{V}(t), \Delta (v))$ of the current state:

$$ C(t, S(t), v(t), \Delta v) \approx \sum_{q=1}^Q \phi _q (t,S(t),\mathcal{V}(t), \Delta (v)) \beta _{q,t} $$

for certain constants $\beta _{q,t} \in \mathbb{R}$ and $Q \in \mathbb{N}$.

First, the coefficients are estimated by a least-squares regression of known continuation values on the current state variables and then a continuation values is approximated by substituting the regression coefficients and a decision rule can be determined.
However, in practice, the continuation values are unknown and need to be approximated. They are dependent on four parameters, but this may be reduced so that the basis functions has dimension one and may be denoted as $\phi(S(t))$, by running separate regressions per volume level and for each time unit.

Thus, the continuation values are approximated for all $M$ independent paths from the simulation by

$$ C^b(t, S^b(t), v(t+1,n)) \approx e^{-\delta} Y^b(t+1, S(t+1), v(t+1,n)) $$

where $Y^b(t+1, S(t+1), v(t+1,n))$ is the accumulated value of future realized cash flows in path $b=1, ..., M$ following optimal decisions starting at time $t+1$ being at volume level $v(t+1)$ and price $S^b (t+1)$. 
Then, the best regression coefficients $\hat{\beta}$ may be estimated by an ordinary least squares regression and the continuation values can be approximated.

With the approximation of the continuation values, a decision for all allowed volume levels can be determined

$$ \hat{\pi}^b (t, S^b(t), v(t)) = \arg \max_{\Delta v \in \mathcal{D}(t,v(t))} \{ h(S^b(t), \Delta v) + \hat{C}^b(t, S^b(t), v(t) + \Delta v)\} $$

that is expected to have the highest accumulated payoff.

The value of future accumulated cash flows is approximated

$$\hat{Y}^b(t+1,S^b(t+1), v(t+1)) = h(S^b(t+1), \hat{\pi}^b(t+1,S^b(t+1),v(t+1))) + e^{-\delta} Y^b(t+2, S^b(t+2), v(t+2) + \hat{\pi}^b(t+1,S^b(t+1),v(t+1)))
$$

Starting at day $T+1$, the continuation value is equal to the penalty function 

$$Y^b(T+1,S^b(T+1), v(T+1)) = q(S^b(T+1),v(T+1))$$

and then the decision rule for all points in the time-volume grid is determined by stepping backwards.

At the valuation date $t=0$, the value of the storage contract is set equal to the average value of the future accumulated cash flows

$$\hat{U}(0, S(0), v(0)) = \frac{1}{M} \sum_{b=1}^M Y^b(1,S^b(1),v(0)) $$